In [3]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from sklearn.metrics import classification_report
import numpy as np
import os

In [4]:
def load_mnist():
    (train_X, train_y), (test_X, test_y) = mnist.load_data()
    train_X = train_X.reshape(-1, 784)
    test_X = test_X.reshape(-1, 784)
    train_X = train_X.astype('float32')
    test_X = test_X.astype('float32')
    train_X /= 255
    test_X /= 255

    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return {"train_X": train_X[:55000, :], "train_y": train_y[:55000, :],
            "val_X": train_X[55000:, :], "val_y": train_y[55000:, :], "test_X": test_X, "test_y": test_y}

In [5]:
def build_network(input_features=None):
    # first we specify an input layer, with a shape == features
    inputs = Input(shape=(input_features,), name="input")
    x = Dense(512, activation='relu', name="hidden1", kernel_regularizer='l2')(inputs)
    x = Dense(256, activation='relu', name="hidden2", kernel_regularizer='l2')(x)
    x = Dense(128, activation='relu', name="hidden3", kernel_regularizer='l2')(x)
    prediction = Dense(10, activation='softmax', name="output")(x)
    model = Model(inputs=inputs, outputs=prediction)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=["accuracy"])
    return model

In [6]:
def create_callbacks():
    tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "tb_log", "mnist_512_256_128_l2"), histogram_freq=1, batch_size=32,
                                       write_graph=True, write_grads=False)
    checkpoint_callback = ModelCheckpoint(filepath="./model-weights.{epoch:02d}-{val_accuracy:.6f}.hdf5", monitor='val_acc',
                                          verbose=0, save_best_only=True)
    return [tensorboard_callback, checkpoint_callback]

In [7]:
def print_model_metrics(model, data):
    loss, accuracy = model.evaluate(x=data["test_X"], y=data["test_y"])
    print("\n model test loss is "+str(loss)+" accuracy is "+str(accuracy))

    y_softmax = model.predict(data["test_X"])  # this is an n x class matrix of probabilities
    y_hat = y_softmax.argmax(axis=-1)  # this will be the class number.
    test_y = data["test_y"].argmax(axis=-1)  # our test data is also categorical
    print(classification_report(test_y, y_hat))

In [8]:
def main():
    data = load_mnist()
    callbacks = create_callbacks()
    model = build_network(data["train_X"].shape[1])
    model.fit(x=data["train_X"], y=data["train_y"],
              batch_size=30,
              epochs=50,
              validation_data=(data["val_X"], data["val_y"]),
              verbose=1,
              callbacks=callbacks)

    print_model_metrics(model, data)

In [9]:
main()

/home/manhcuong/anaconda3/envs/python3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/home/manhcuong/anaconda3/envs/python3.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


Epoch 1/50
1834/1834 [==============================] - 19s 10ms/step - loss: 1.1410 - accuracy: 0.8859 - val_loss: 0.6038 - val_accuracy: 0.9248
Epoch 2/50
1834/1834 [==============================] - 17s 9ms/step - loss: 0.5786 - accuracy: 0.9225 - val_loss: 0.4982 - val_accuracy: 0.9442
Epoch 3/50
1834/1834 [==============================] - 17s 9ms/step - loss: 0.4973 - accuracy: 0.9336 - val_loss: 0.4217 - val_accuracy: 0.9560
Epoch 4/50
1834/1834 [==============================] - 17s 9ms/step - loss: 0.4560 - accuracy: 0.9376 - val_loss: 0.3979 - val_accuracy: 0.9538
Epoch 5/50
1834/1834 [==============================] - 17s 10ms/step - loss: 0.4294 - accuracy: 0.9404 - val_loss: 0.3668 - val_accuracy: 0.9552
Epoch 6/50
1834/1834 [==============================] - 17s 9ms/step - loss: 0.4116 - accuracy: 0.9439 - val_loss: 0.3635 - val_accuracy: 0.9608
Epoch 7/50
1834/1834 [==============================] - 17s 9ms/step - loss: 0.3971 - accuracy: 0.9452 - val_loss: 0.3425 - val_